In [0]:

CREATE OR REPLACE TABLE inceptez_catalog.inputdb.tblelectiondata USING DELTA AS
SELECT
 *
FROM
  csv.`dbfs:/Volumes/inceptez_catalog/inputdb/electiondata/election_data_2014.csv` WITH (inferSchema = "true", header = "true");

  select * from inceptez_catalog.inputdb.tblelectiondata;

  describe table inceptez_catalog.inputdb.tblelectiondata;

with electiondata as
  (select *, row_number() over(partition by constituency order by total desc) as rnk from inceptez_catalog.inputdb.tblelectiondata)
  select partyname,count(*) as win_candidate_cnt from electiondata where rnk = 1 group by partyname order by 2 desc;

with electiondata as
  (select *, row_number() over(partition by constituency order by total desc) as rnk from inceptez_catalog.inputdb.tblelectiondata)
  select *, case when rnk = 1 then 'Won' else 'Lost' end as result from electiondata;


alter table inceptez_catalog.inputdb.tblelectiondata add columns (result string);

insert overwrite inceptez_catalog.inputdb.tblelectiondata
with electiondata as
  (select *, row_number() over(partition by constituency order by total desc) as rnk from inceptez_catalog.inputdb.tblelectiondata)
  select state,
constituency,
candidate_name,
sex,
age,
category,
partyname,
partysymbol,
general,
postal,
total,
pct_of_total_votes,
pct_of_polled_votes,
totalvoters, 
case when rnk = 1 then 'Won' else 'Lost' end as result from electiondata;


select * from inceptez_catalog.inputdb.tblelectiondata where result='Won';
